In [86]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [87]:

internal_urls = set()
external_urls = set()

In [88]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [89]:

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls


In [90]:
total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [91]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.berry.edu/student-life/life-on-campus/health-center/updates/"
internal_urls.add(base_url)

In [92]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='www.berry.edu', path='/student-life/life-on-campus/health-center/updates/', params='', query='', fragment='')
https://www.berry.edu
www.berry.edu/student-life/life-on-campus/health-center/updates/
www.berry.edu


In [93]:
# internal_urls

In [94]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

[INT] Internal link: https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
[INT] Internal link: https://www.berry.edu/default.aspx
[INT] Internal link: https://www.berry.edu/admission/pay-deposit.aspx
[INT] Internal link: https://www.berry.edu/admission/how-to-apply/default.aspx
[INT] Internal link: https://www.berry.edu/academics/majors/default.aspx
[INT] Internal link: https://www.berry.edu/giving/default.aspx
[INT] Internal link: https://www.berry.edu/about/default.aspx
[INT] Internal link: https://www.berry.edu/about/quick-facts.aspx
[INT] Internal link: https://www.berry.edu/about/our-rich-history/default.aspx
[INT] Internal link: https://www.berry.edu/about/the-berry-compact.aspx
[INT] Internal link: https://www.berry.edu/about/culture-of-mentors.aspx
[INT] Internal link: https://www.berry.edu/about/student-work.aspx
[INT] Internal link: https://www.berry.edu/about/campus.aspx
[INT] Internal link: https://www.berry.edu/about/purpose-and-mission.aspx
[IN

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.berry.edu/alumni-parents-friends/_assets/services-for-alumni-interest-groups.pdf
[INT] Internal link: https://www.berry.edu/alumni/golden-guard.aspx
[INT] Internal link: https://www.berry.edu/alumni/daughters-of-berry.aspx
[INT] Internal link: https://www.berry.edu/alumni/berry-breakfast-club.aspx
[INT] Internal link: https://www.berry.edu/alumni/handicrafts-network.aspx
[INT] Internal link: https://www.berry.edu/alumni/berry-alumni-quilters.aspx


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.berry.edu/alumni/welcome.aspx
[INT] Internal link: https://www.berry.edu/alumni/_images/Cameron-Newman.jpg
[INT] Internal link: https://www.berry.edu/alumni/_images/Jetta.jpg
[INT] Internal link: https://www.berry.edu/alumni/_images/Bailey2.jpg


...

[INT] Internal link: https://www.berry.edu/academics/fs/_pdf/SniderCV.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.berry.edu/academics/fs/_pdf/Daniel CV.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.berry.edu/academics/fs/_pdf/Adams CV.pdf
[+] Total External links: 439
[+] Total Internal links: 444
[+] Total: 883


In [95]:
count = 0
last_count = 989874
immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

for i in range(0,4):
    for url in internal_urls:
        if base_url_text in url:
            immediate_urls.add(url)
    count = len(immediate_urls)
    print(count,i)
    if(count == last_count):
        break
    last_count = count
    for immediate_url in immediate_urls:
        try:
            crawl(immediate_url)
            print("[+] NEW Total External links:", len(external_urls))
            print("[+] NEW Total Internal links:", len(internal_urls))
            print("[+] NEW Total:", len(external_urls) + len(internal_urls))
            if len(internal_urls)>1000:
                break
        except requests.exceptions.ConnectionError:
            continue

6 0
[INT] Internal link: https://www.berry.edu/student-life/life-on-campus/health-center/updates/default.aspx
[INT] Internal link: https://www.berry.edu/academics/_assets/Academic-Calendar_2020-21.pdf
[INT] Internal link: https://www.berry.edu/fallopening
[+] NEW Total External links: 446
[+] NEW Total Internal links: 447
[+] NEW Total: 893
[INT] Internal link: https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
[+] NEW Total External links: 446
[+] NEW Total Internal links: 448
[+] NEW Total: 894
[INT] Internal link: https://www.berry.edu/student-life/life-on-campus/health-center/updates/%20https:/www.cdc.gov/coronavirus/2019-ncov/travelers/index.html
[+] NEW Total External links: 452
[+] NEW Total Internal links: 449
[+] NEW Total: 901
[INT] Internal link: http://www.berry.edu/student-life/life-on-campus/health-center/default.aspx
[INT] Internal link: http://www.berry.edu/student-life/life-on-campus/health-center/updates/default.aspx
[INT] Intern

[+] NEW Total External links: 486
[+] NEW Total Internal links: 561
[+] NEW Total: 1047
[+] NEW Total External links: 486
[+] NEW Total Internal links: 561
[+] NEW Total: 1047
[+] NEW Total External links: 486
[+] NEW Total Internal links: 561
[+] NEW Total: 1047
[INT] Internal link: http://www.berry.edu/student-life/life-on-campus/fall-opening/faq.aspx
[INT] Internal link: http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-archive.aspx
[INT] Internal link: http://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
[+] NEW Total External links: 486
[+] NEW Total Internal links: 564
[+] NEW Total: 1050
[+] NEW Total External links: 486
[+] NEW Total Internal links: 564
[+] NEW Total: 1050
14 2
[+] NEW Total External links: 486
[+] NEW Total Internal links: 564
[+] NEW Total: 1050
[+] NEW Total External links: 486
[+] NEW Total Internal links: 564
[+] NEW Total: 1050
[+] NEW Total External links: 486
[+] NEW Total Internal links: 564
[+

[+] NEW Total External links: 486
[+] NEW Total Internal links: 681
[+] NEW Total: 1167
[+] NEW Total External links: 486
[+] NEW Total Internal links: 681
[+] NEW Total: 1167
[+] NEW Total External links: 486
[+] NEW Total Internal links: 681
[+] NEW Total: 1167
[+] NEW Total External links: 486
[+] NEW Total Internal links: 681
[+] NEW Total: 1167
[+] NEW Total External links: 486
[+] NEW Total Internal links: 681
[+] NEW Total: 1167


In [96]:
def removeAfterN(yourStr, nth, occurenceOf):
    return occurenceOf.join(yourStr.split(occurenceOf)[:nth])

In [97]:
semifinal_immediate_urls = set()

for immediate_url in immediate_urls:
    stripped_url = removeAfterN(immediate_url,2,"http")
    
    
    semifinal_immediate_urls.add(stripped_url)
#     final_immediate_urls.add(rest)
#     print(rest)


    
for i in semifinal_immediate_urls:
    print(i)
total_count = len(semifinal_immediate_urls)
print(total_count)

https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students
https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-archive.aspx
http://www.berry.edu/student-life/life-on-campus/health-center/updates/%20
http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updates/%20
http://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
http://www.berry.edu/student-life/life-on-campus/health-center/updates/update-archive.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updat

In [98]:
final_immediate_urls = set()
for semifinal_immediate_url in semifinal_immediate_urls:
    if "tel:" not in semifinal_immediate_url:
        final_immediate_urls.add(semifinal_immediate_url)


for i in final_immediate_urls:
    print(i)
total_count = len(final_immediate_urls)
print(total_count)       

https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students
https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-archive.aspx
http://www.berry.edu/student-life/life-on-campus/health-center/updates/%20
http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updates/%20
http://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
http://www.berry.edu/student-life/life-on-campus/health-center/updates/update-archive.aspx
https://www.berry.edu/student-life/life-on-campus/health-center/updat

In [99]:

parent_folder = "Masters" #Change based on folder
name_of_folder = f'{base_url_text_domain}_Screenshots'
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
    os.makedirs(f'{parent_folder}\\{name_of_folder}')

In [100]:
webpage_save = set()
webpage_save_count = 0
webpage_cannot_save = 0
for i in final_immediate_urls:
    if ' ' in i:
        continue
    try:
        code=urlopen(i).getcode()
        webpage_save.add(i)
        webpage_save_count = webpage_save_count+1
        print(f"{code} can_save - {i}")
    except (urllib.error.HTTPError,urllib.error.URLError) as e:
        print("discarded")
        webpage_cannot_save = webpage_cannot_save+1
        continue

print(f'Webpages save as HTML :{webpage_save_count}')
print(f'Webpages unable to save as HTML :{webpage_cannot_save}')
        

200 can_save - https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students
200 can_save - https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
200 can_save - https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
200 can_save - https://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
200 can_save - https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-archive.aspx
discarded
200 can_save - http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
200 can_save - http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
discarded
200 can_save - http://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
200 can_save - http://www.berry.edu/student-life/life-on-campus/health-center/updates/update-archive.aspx
200 can_save - https://www.berry.edu/student-life/life-on-campus/

In [101]:
pdf_count = 0
pdf_urls = set()
for webpage in webpage_save:
    try:
        if webpage[-4:] == ".pdf":
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                file_name = "Home"
            pdf_urls.add(webpage)
            print(webpage)
            r = requests.get(url, stream=True)
            with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                for chunk in r.iter_content(2000):
                    fd.write(chunk)
            pdf_count = pdf_count + 1
    except:
        print("caught")

print(f"Pdfs saved: {pdf_count}")

Pdfs saved: 0


In [102]:
final_immediate_urls = final_immediate_urls - pdf_urls
webpage_save = webpage_save - pdf_urls
web_save_count = len(webpage_save)
total_count = len(final_immediate_urls)
print(total_count) 

16


In [103]:

#     try:
#         print(i)
        
#         print("added")
#     except urllib.error.HTTPError as e:
#         if e.code in (400,403):
#             print("discarded")
#             continue

In [104]:
final_webpage_save_count = webpage_save_count - pdf_count
print(final_webpage_save_count)

14


In [105]:
webpage_save_counter_loop = 0
for webpage in webpage_save:
    try:
        url = webpage
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
        webpage_save_counter_loop = webpage_save_counter_loop+1
        print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
    except:
        print("caught")

print("Saving Completed")

1 of 14 - https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students
2 of 14 - https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
3 of 14 - https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
4 of 14 - https://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
5 of 14 - https://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-archive.aspx
6 of 14 - http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq-students.aspx
7 of 14 - http://www.berry.edu/student-life/life-on-campus/health-center/updates/faq.aspx
8 of 14 - http://www.berry.edu/student-life/life-on-campus/health-center/updates/what-to-do.aspx
9 of 14 - http://www.berry.edu/student-life/life-on-campus/health-center/updates/update-archive.aspx
10 of 14 - https://www.berry.edu/student-life/life-on-campus/health-center/updates/default.aspx
11 of 14 - https://www.berry.edu/s

In [106]:
count = 0
for i in final_immediate_urls:
    try:
        url = i
        file_name = url.replace(base_url,'')
        file_name = file_name.replace('/','')
        if file_name == '':
                file_name = "Home"
        count = count + 1
        print(f"{count} of {total_count}")
        print(f'Visiting {base_url_text}/{file_name}')
        print(f'...Taking a screenshot')
        driver.get(url)
        S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
        driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
        driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
        print(f'Screenshot of {file_name} page taken! \n')
    except:
        print("caught")
#     print("Saving HTML")
#     pyautogui.hotkey('ctrl', 's')
#     time.sleep(1)
#     pyautogui.typewrite(f'{file_name}.html')
#     pyautogui.hotkey('enter')
driver.quit()
    
print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

1 of 16
Visiting www.berry.edu/student-life/life-on-campus/health-center/updates//faq-students
...Taking a screenshot
caught
2 of 16
Visiting www.berry.edu/student-life/life-on-campus/health-center/updates//faq-students.aspx
...Taking a screenshot
caught
3 of 16
Visiting www.berry.edu/student-life/life-on-campus/health-center/updates//faq-archive.aspx
...Taking a screenshot
caught
4 of 16
Visiting www.berry.edu/student-life/life-on-campus/health-center/updates//http:www.berry.edustudent-lifelife-on-campushealth-centerupdatesfaq-students.aspx
...Taking a screenshot
caught
5 of 16
Visiting www.berry.edu/student-life/life-on-campus/health-center/updates//http:www.berry.edustudent-lifelife-on-campushealth-centerupdatesfaq.aspx
...Taking a screenshot
caught
6 of 16
Visiting www.berry.edu/student-life/life-on-campus/health-center/updates//http:www.berry.edustudent-lifelife-on-campushealth-centerupdateswhat-to-do.aspx
...Taking a screenshot
caught
7 of 16
Visiting www.berry.edu/student-life/l